In [27]:
import spacy

# change this with your text example
file_path = 'news.txt'

with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

nlp = spacy.load('en_core_web_lg') 

doc = nlp(text) 

spacy.displacy.render(doc, style='ent')

entities = [(ent.text, ent.label_) for ent in doc.ents]





In [46]:
from rdflib import Graph, RDF, RDFS, OWL

# Load the existing ontology
existing_ontology = Graph()
# Change the test.ttl with your own ontology if needed you can also change the format
existing_ontology.parse("test.ttl", format="turtle")

matching_classes = 0
matching_instances = 0
total_classes = 0
total_instances = 0

# Lists to store instances and classes that are different
different_classes = []
different_instances = []
# Lists to make sure no duplicate classes (for count purposes)
nodup_classes = []

# Iterate through the generated ontology entities
for entity, label in entities:
    # Encode the entity name to create a valid URI
    encoded_entity = entity.replace(' ', '_')
    encoded_entity = encoded_entity.replace('"', '')
    entity_uri = URIRef(f"http://example.com/entities/{encoded_entity}")

    # Check if the class exists in the existing ontology
    class_uri = URIRef(f"http://example.com/classes/{label}")
    if (class_uri, RDF.type, OWL.Class) in existing_ontology:
        if label not in nodup_classes:
            nodup_classes.append(f"{label}")
            matching_classes += 1
    else:
        # Class is different
        different_classes.append(f"Class '{label}'")

    # Check if the entity exists in the existing ontology through URI or label matching
    if (entity_uri, RDF.type, None) not in existing_ontology and (entity_uri, RDFS.label, Literal(entity)) not in existing_ontology :
        # Entity does not exist in the existing ontology
        different_instances.append(f"Instance '{entity}' is not found in the existing ontology.")
    elif (entity_uri, RDF.type, class_uri) not in existing_ontology :
        different_instances.append(f"Instance '{entity}' with class '{label} is not found in the existing ontology.")    
    else:
        # Entity exists in the existing ontology 
        matching_instances += 1
        
    total_instances += 1


# Print the results
print(f"Matching classes: {matching_classes}/{len(nodup_classes) + len(different_classes)}")

print(f"Matching instances: {matching_instances}/{total_instances}")

if different_classes:
    print("Classes that cant be found in ontology (coming from the text):")
    for class_diff in different_classes:
        print(f"- {class_diff}")

if different_instances:
    print("Instances that cant be found in ontology (coming from the text):")
    for instance_diff in different_instances:
        print(f"- {instance_diff}")


Matching classes: 7/8
Matching instances: 14/21
Classes that cant be found in ontology (coming from the text):
- Class 'PRODUCT'
Instances that cant be found in ontology (coming from the text):
- Instance 'Fat Bear Week' with class 'ORG is not found in the existing ontology.
- Instance 'one' is not found in the existing ontology.
- Instance 'Brooks River' with class 'ORG is not found in the existing ontology.
- Instance 'Grazer' with class 'PRODUCT is not found in the existing ontology.
- Instance 'Katmai National Park' with class 'ORG is not found in the existing ontology.
- Instance 'Facebook' is not found in the existing ontology.
- Instance 'Chin up' is not found in the existing ontology.
